# Missing Data Analysis (Mechanism Test)

### **Objective**
Before we fix the missing data, we need to categorize the "Missingness Mechanism." 

**Crucial Note:** We can only statistically **prove** Missing at Random (MAR) by finding correlations in the data. For the other types (MNAR and MCAR), we infer them based on the evidence (or lack of it).

### **The Three Categories:**

1.  **Likely MAR (Missing at Random):**
    * **The Test:** We found a statistical correlation (`r > 0.1`) between the missing values and other answers.
    * **Conclusion:** **PROVEN.** The missingness is tied to a pattern we can see. We will use this pattern to Impute.

2.  **Potential MCAR (Missing Completely at Random):**
    * **The Test:** We tested for patterns but found **zero correlation**.
    * **Conclusion:** **ASSUMED.** Since there is no link to other data, we assume it is random error (MCAR). It is safe to impute.

3.  **Potential MNAR (Missing Not at Random):**
    * **The Test:** The data is **> 50% Missing**.
    * **Conclusion:** **INFERRED.** There is too little data to run a reliable test. We assume this is "Structural" (e.g., the question didn't apply). We generally Drop these to be safe.

### **Output**
This notebook will generate a CSV report (`missingness_diagnostic_report.csv`) saved in the `data/interim/diagnostics/` folder. This report will be used as the "Instruction Manual" for the imputation step.

# Config Loader

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# CONFIGURATION & SETUP
def load_config():
    """
    Loads project configuration to locate the dataset.
    """
    try:
        config_path = Path("data/interim/config.json")
        if config_path.exists():
            with open(config_path) as f:
                cfg = json.load(f)
            return Path(cfg["BASE_PATH"])
        else:
            return Path("../")
    except:
        return Path("../")

BASE_PATH = load_config()
INPUT_DIR = BASE_PATH / "NEW Renamed Fully Decoded Surveys V2" 
files = list(INPUT_DIR.rglob("*.CSV"))

print(f"Target Directory: {INPUT_DIR}")
print(f"Files for Analysis: {len(files)}")

Target Directory: G:\My Drive\Labor Force Survey\NEW Renamed Fully Decoded Surveys V2
Files for Analysis: 40


# Statistical Pattern Test

In [2]:
# STATISTICAL PATTERN TEST
import warnings
import os

# SILENCER
warnings.filterwarnings('ignore')

# SETUP OUTPUT: "Missingness Reports" in your G-Drive
# We use the BASE_PATH you defined in the previous cell.
# This creates a folder next to your survey data folder.
OUTPUT_DIR = BASE_PATH / "Missingness Reports"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE = OUTPUT_DIR / "missingness_diagnostic_report.csv"

diagnostic_results = []

print(f"Running Diagnostics... (Optimized Mode)")
print(f"Saving Report To: {OUTPUT_FILE}")
print("-" * 60)

for file_path in files:
    try:
        # 1. Load Data
        df = pd.read_csv(file_path, sep=None, engine='python', 
                         na_values=[' ', '', 'N/A', 'NULL', '.', 'nan', 'no response'])
        df.columns = [c.strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
        
        miss_pct = df.isnull().mean()
        missing_vars = miss_pct[miss_pct > 0].sort_values(ascending=False)
        
        if len(missing_vars) == 0:
            continue

        # Convert to numbers ONCE per file
        numeric_df = df.apply(lambda x: pd.factorize(x)[0])

        # 2. Loop through variables
        for var, pct in missing_vars.items():
            pct_val = pct * 100
            test_result = ""
            verdict = ""
            r_value = 0.0 
            
            # A. STRUCTURAL / DROP ZONE (>50%)
            if pct > 0.50:
                test_result = "Insufficient Data"
                verdict = "Potential MNAR (Drop)"
                r_value = np.nan
            
            # B. TESTING ZONE (<=50%)
            else:
                try:
                    target_indicator = df[var].isnull().astype(int)
                    
                    # Calculate correlation
                    correlations = numeric_df.corrwith(target_indicator).abs()
                    
                    if var in correlations:
                        correlations = correlations.drop(var)
                    
                    max_corr = correlations.max()
                    r_value = max_corr
                    
                    if max_corr >= 0.1:
                        test_result = f"Pattern Found (r={max_corr:.2f})"
                        verdict = "Likely MAR (Impute)"
                    else:
                        test_result = "No Pattern Found"
                        verdict = "Potential MCAR (Impute)"
                        
                except Exception:
                    test_result = "Calculation Error"
                    verdict = "Check Manually"
                    r_value = np.nan

            # Append results
            diagnostic_results.append({
                "filename": file_path.name,
                "variable": var,
                "missing_percent": round(pct_val, 2),
                "test_result": test_result,
                "r_value": round(r_value, 4) if pd.notna(r_value) else None,
                "verdict": verdict
            })

    except Exception as e:
        print(f"Error processing {file_path.name}: {str(e)}")

# Save and View
if diagnostic_results:
    results_df = pd.DataFrame(diagnostic_results)
    results_df.to_csv(OUTPUT_FILE, index=False)
    
    print("-" * 60)
    print(f"SUCCESS: Report saved.")
    print(f"Total variables diagnosed: {len(results_df)}")
    print("-" * 60)
    
    # Show clean sample
    display(results_df.head(20))
else:
    print("\nNo missing data found.")

Running Diagnostics... (Optimized Mode)
Saving Report To: G:\My Drive\Labor Force Survey\Missingness Reports\missingness_diagnostic_report.csv
------------------------------------------------------------
------------------------------------------------------------
SUCCESS: Report saved.
Total variables diagnosed: 1326
------------------------------------------------------------


,filename,variable,missing_percent,test_result,r_value,verdict
0,JANUARY_2024.CSV,first_time_to_look_for_work,98.94,Insufficient Data,NaN,Potential MNAR (Drop)
1,JANUARY_2024.CSV,c33_number_of_weeks_spent_in_looking_for_work,98.94,Insufficient Data,NaN,Potential MNAR (Drop)
2,JANUARY_2024.CSV,c32_job_search_method,98.94,Insufficient Data,NaN,Potential MNAR (Drop)
3,JANUARY_2024.CSV,c35_when_last_looked_for_work,98.82,Insufficient Data,NaN,Potential MNAR (Drop)
4,JANUARY_2024.CSV,c27_number_of_jobs_during_the_past_week,96.61,Insufficient Data,NaN,Potential MNAR (Drop)
5,JANUARY_2024.CSV,c37_willingness_to_take_up_work_during_the_pas...,96.41,Insufficient Data,NaN,Potential MNAR (Drop)
6,JANUARY_2024.CSV,available_for_work,96.41,Insufficient Data,NaN,Potential MNAR (Drop)
7,JANUARY_2024.CSV,c39___last_worked_(year),87.98,Insufficient Data,NaN,Potential MNAR (Drop)
8,JANUARY_2024.CSV,kind_of_business_(past_quarter),87.98,Insufficient Data,NaN,Potential MNAR (Drop)
9,JANUARY_2024.CSV,c41__previous_occupation,87.98,Insufficient Data,NaN,Potential MNAR (Drop)
